# **TENTS FINAL**

### **Rules**

##### Original Rules

- There are exactly as many tents as trees.

- The tents and trees can be matched up in such a way that each tent is directly adjacent (horizontally or vertically, but not diagonally) to its own tree. However, a tent may be adjacent to other trees as well as its own.

- No two tents are adjacent horizontally, vertically or diagonally.

- The number of tents in each row, and in each column, matches the numbers given round the sides of the grid.

##### Additional Rules

- For tents, they have to have a space a land next to them and a tree.

- However for the tree, they have to have a water by them, for them to grow. A tent cannot be surrounded by water because they need to step out of it.

- Water must be all connected and orthogonally.


### **What it would look like**

This is a solved puzzle, with a minor adjustment with how this puzzle was generated.

Below I have also coded the puzzle, T is a tent, R are trees, and W is water.


<img src="unsolved_trees.png" alt="Alt Text" width="300">
<img src="solved_trees.png" alt="Alt Text" width="300">
<img src="solved_trees_with_water.png" alt="Alt Text" width="300">


In [231]:
from itertools import permutations

In [205]:
blank = [['0' for i in range(8)] for i in range(8)]
blank

[['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0']]

In [206]:
blank[0][6] = 'R'
blank[1][1] = 'R'
blank[2][0] = 'R'
blank[2][5] = 'R'
blank[4][0] = 'R'
blank[4][7] = 'R'
blank[5][4] = 'R'
blank[5][7] = 'R'
blank[7][0] = 'R'
blank[7][2] = 'R'
blank[7][6] = 'R'

In [207]:
blank[0][1] = 'W'
blank[0][2] = 'W'
blank[0][3] = 'W'
blank[0][4] = 'W'
blank[0][5] = 'W'
blank[1][4] = 'W'
blank[2][4] = 'W'
blank[3][4] = 'W'
blank[3][0] = 'W'
blank[3][1] = 'W'
blank[3][2] = 'W'
blank[3][3] = 'W'
blank[3][5] = 'W'
blank[3][6] = 'W'
blank[4][3] = 'W'
blank[4][6] = 'W'
blank[5][3] = 'W'
blank[6][1] = 'W'
blank[6][2] = 'W'
blank[6][3] = 'W'
blank[6][4] = 'W'
blank[6][5] = 'W'
blank[6][6] = 'W'
blank[6][7] = 'W'
blank[7][1] = 'W'

In [208]:
blank[0][7] = 'T'
blank[1][0] = 'T'
blank[1][2] = 'T'
blank[1][5] = 'T'
blank[3][7] = 'T'
blank[4][1] = 'T'
blank[4][4] = 'T'
blank[5][6] = 'T'
blank[6][0] = 'T'
blank[7][3] = 'T'
blank[7][5] = 'T'

In [209]:
blank

[['0', 'W', 'W', 'W', 'W', 'W', 'R', 'T'],
 ['T', 'R', 'T', '0', 'W', 'T', '0', '0'],
 ['R', '0', '0', '0', 'W', 'R', '0', '0'],
 ['W', 'W', 'W', 'W', 'W', 'W', 'W', 'T'],
 ['R', 'T', '0', 'W', 'T', '0', 'W', 'R'],
 ['0', '0', '0', 'W', 'R', '0', 'T', 'R'],
 ['T', 'W', 'W', 'W', 'W', 'W', 'W', 'W'],
 ['R', 'W', 'R', 'T', '0', 'T', 'R', '0']]

In [ ]:
def get_position(board):
  positions = {}
  for i, row in enumerate(board):
    for j, item in enumerate(row):
      if item in positions.keys():
        positions[item].append((i,j))
      else:
        positions[item] = [(i,j)]
  return positions

def connecting_positions(pos, diagonal=False):
  x, y = pos[0], pos[1]
  possible_positions = [(x+1,y), (x,y+1), (x-1, y), (x, y-1)]
  if diagonal:
    possible_positions.extend([(x-1, y+1), (x-1, y-1), (x+1, y+1), (x+1, y-1)])
  return possible_positions

def dfs(node, connections, visited):
  visited.add(node)
  for position in connections.get(node):
    if position not in visited:
      dfs(position, connections, visited)

def is_connected(connections):
  visit = set()

  dfs(next(iter(connections)), connections, visit)

  return len(visit) == len(connections)

def tree_and_tent(connections):
  visit = set()

  count = 0
  while len(visit) != len(connections):
    for connection in connections.values():
      if len(connection) == 1:
        visit.add(connection[0])
      else:
        for pos in connection:
          if pos in visit:
            connection.remove(pos)
        if len(connection) == 1:
          visit.add(connection[0])
      count += 1
      if count > 300:
        return False
  return True

def get_view(board, row=0, column=0):
  if row:
    return board[row-1]
  elif column:
    column_view = []
    for row in board:
      column_view.append(row[column-1])
    return column_view
  
def get_hints(board):
  hints = {}
  hints['row'] = {}
  hints['column'] = {}
  hints['row']['trees'] = []
  hints['column']['trees'] = []
  hints['row']['water'] = []
  hints['column']['water'] = []
  y = len(board)
  x = len(board[0])
  for i in range(1, x+1):
    view = get_view(board, row=i)
    tree_count = 0
    water_count = 0
    for item in view:
      if item == 'T':
        tree_count += 1
      if item == 'W':
        water_count += 1
    hints['row']['trees'].append(tree_count)
    hints['row']['water'].append(water_count)
  for i in range(1, y):
    view = get_view(board, column=i)
    tree_count = 0
    water_count = 0    
    for item in view:
      if item == 'T':
        tree_count += 1
      if item == 'W':
        water_count += 1
    hints['column']['trees'].append(tree_count)
    hints['column']['water'].append(water_count)
  
  return hints


def check_valid_solution(board, hints):
  positions = get_position(board)
  valid = len(positions['R']) == len(positions['T'])
  if valid:
    pass
  else:
    print("There are not an equal amount of trees and tents")
    return False


  # Check for all trees have a water source
  for tree in positions['R']:
    for orth_tree in connecting_positions(tree):
      if orth_tree in positions['W']:
        valid = True
        break
      else:
        valid = False
    if valid == False:
      print("False at tree position", tree)
      return False
  
  # Check for all tents having open land
  for tent in positions['T']:
    for orth_tent in connecting_positions(tent):
      if orth_tent in positions['0']:
        valid = True
        break
      else:
        valid = False
    if valid == False:
      print("False at tent position", tent)
      return False
  
  # check for all tents not being next to each other
  for tent in positions['T']:
    for pos_tent in connecting_positions(tent, True):
      if pos_tent in positions['T']:
        valid = False
        print(f"Found two tents next to each other at positions {tent} and {pos_tent}")
        return False
      else:
        valid = True

  # Check if water is orthogonally connected
  water_connections = {}
  water_positions = positions['W'].copy()
  for water in positions['W']:
    temp_connections = []
    for orth_water in connecting_positions(water):
      if orth_water in water_positions:
        temp_connections.append(orth_water)
    water_connections[water] = temp_connections
  valid = is_connected(water_connections)

  # Check if each tent has a unique tree
  tree_tent_connections = {}
  for tent in positions['T']:
    tree_tent_connections[tent] = []
    for orth_tree in connecting_positions(tent):
      if orth_tree in positions['R']:
        tree_tent_connections[tent].append(orth_tree)
  valid = tree_and_tent(tree_tent_connections)

  # Check if hints match up with puzzle
  board_hints = get_hints(board)
  valid = board_hints == hints

  return valid

In [235]:
hints = get_hints(blank)

In [257]:
row_clues = {}
column_clues = {}
for i in range(8):
  
  row_clue = ""
  row_clue += 'T' * hints['row']['trees'][i]
  row_clue += "W" * hints['row']['water'][i]
  row_zeroes = 8 - len(row_clue)
  row_clue += "0" * row_zeroes
  row_clues[i] = row_clue

  column_clue = ""
  column_clue += 'T' * hints['row']['trees'][i]
  column_clue += "W" * hints['row']['water'][i]
  column_zeroes = 8 - len(column_clue)
  column_clue += "0" * column_zeroes
  column_clues[i] = column_clue


In [255]:
row_perms = {}
for key, item in row_clues.items():
  row_perm = permutations(item, 8)
  row_perms[key] = set()
  for p in row_perm:
    row_perms[key].add(p)

column_perms = {}
for key, item in column_clues.items():
  column_perm = permutations(item, 8)
  column_perms[key] = set()
  for p in column_perm:
    column_perms[key].add(p)

In [ ]:
def backtrack():
  pass

In [ ]:
def place_tree_and_tent():
  pass

def generate_new_puzzle( ):
  trees_and_tents = 11
  x = 8
  y = 8
  blank_board=  [['0' for i in range(x)] for j in range(y)]
  pass